# Distributed Isolation Forest

In this project we implemented a distributed versino of the Isolation Forest algorithm.
We compare our implementation to 3 other implementations:
1. Sklearn-IForest - non distributed version of Isolation Forest
2. spark-iforest - a distributed implementation of Isolation Forest
3. SynapseML - distributed implementation by Microsoft

We compare the performance on the following datasets:
1. Shuttle
2. ForestCover
3. Annthyroid
4. Arrhythmia
5. Mammography
6. Http (KDDCUP99)

## Imports

In [0]:
import sys
print("\n".join(sys.path))

/databricks/python_shell/scripts
/local_disk0/spark-90c0bb8b-eb54-4d38-9b69-6a23529895de/userFiles-a0812170-c3f3-4234-869d-fbe887f35900/addedFile2372871097350889815com_microsoft_azure_synapseml_vw_2_12_0_10_2-81557.jar
/local_disk0/spark-90c0bb8b-eb54-4d38-9b69-6a23529895de/userFiles-a0812170-c3f3-4234-869d-fbe887f35900/addedFile1755398803738913152com_microsoft_azure_synapseml_opencv_2_12_0_10_2-d58bb.jar
/local_disk0/spark-90c0bb8b-eb54-4d38-9b69-6a23529895de/userFiles-a0812170-c3f3-4234-869d-fbe887f35900/addedFile364942136124539941com_microsoft_azure_synapseml_lightgbm_2_12_0_10_2-26d77.jar
/local_disk0/spark-90c0bb8b-eb54-4d38-9b69-6a23529895de/userFiles-a0812170-c3f3-4234-869d-fbe887f35900/addedFile8584233862606803279com_microsoft_azure_synapseml_deep_learning_2_12_0_10_2-35ed1.jar
/local_disk0/spark-90c0bb8b-eb54-4d38-9b69-6a23529895de/userFiles-a0812170-c3f3-4234-869d-fbe887f35900/addedFile6972968937420700704com_microsoft_azure_synapseml_cognitive_2_12_0_10_2-799f1.jar
/local_dis

In [0]:
from pyspark.ml.feature import VectorAssembler
from sklearn import metrics
from DIForest import DIForest
import time

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-3701505335330571> in <cell line: 3>()
      1 from pyspark.ml.feature import VectorAssembler
      2 from sklearn import metrics
----> 3 from DIForest import DIForest
      4 import time

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    169             # Import the desired module. If you’re seeing this while debugging a failed import,
    170             # look at preceding stack frames for relevant error information.
--> 171             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    172 
    173             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'DIForest'

## Utils

In [0]:
def evaluate_model(model_name, y_true, y_pred, y_score):
    print("Accuracy Score :")
    print(metrics.accuracy_score(y_true, y_pred))
    print()
    
    print("Classification Report :")
    print(metrics.classification_report(y_true, y_pred))
    print()
    
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(y_true, y_pred))
    print()
    
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
    auc = metrics.roc_auc_score(y_true, y_score)
    
    plt.subplots(1, figsize=(10,10))
    plt.title(f'{model_name}, AUC={auc}')
    plt.plot(fpr, tpr)
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

## Shuttle Dataset

In [0]:
data = spark.read.table("hive_metastore.default.shuttle")
data.printSchema()

samples_count = data.count()
outliers_count = data.where(data["Y"] == 1).count()

print("Count: ", samples_count)
print("Outliers: ", outliers_count)
print("Outliers percentage: ", outliers_count * 100 / samples_count, "%")

input_cols=["attr1", "attr2", "attr3", "attr4", "attr5", "attr6", "attr7", "attr8", "attr9"]

assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
data = assembler.transform(data)

### Our implementation

In [0]:
start_time = time.time()

dis_model = DIForest(100, 256).fit(spark, data)
predictions = dis_model.transform(spark, data)

et = time.time()

elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

evaluate_model("DISForest", predictions_df["Y"], predictions_df["prediction"], predictions_df["anomalyScore"])